# Criptografía - Práctica 3: Funciones de un solo sentido
Guillermo Chica Sabariego

### Ejercicio 1: Función mochila

En este ejercicio se nos pide implementar un cifrado de mochila de clave asimétrica donde la clave privada es una lista super creciente y dos enteros *n* y *u* primos entre sí y la clave pública es una versión modificada de la lista supercreciente obtenida multiplicando cada elemento de la lista supercreciente por *u* mod *n*.

Para cifrar, cada bit del mensaje se multiplica por un elemento de la lista supercreciente que es la llave pública y se suma cada uno de estos productos, obteniéndose un número. Para descifar, se calcula la sumatoria del producto de cada bit del mensaje con la lista super creciente y descomponemos este número como suma de los elementos de la lista super creciente.

Paso a paso, primero, implementamos las funciones de MCD e inverso de la prácica 1 que nos servirán en esta:

In [14]:
#******************************************************
def mcd(a,b):
    """
    Implementación del algoritmo de euclides para el cálculo del MCD y los
    coeficientes de Bezout
    
    Entrada: 
        a, b: enteros 
    
    Salida:
        El mcd y los coeficientes u y b
    """


    #Inicilizamos u y v para a y b
    u_a=1
    v_a=0
    u_b=0
    v_b=1

    while(b!=0):
        #Hallamos cociente
        c = a / b
        #Valores futuros de u y v
        aux_u = u_a - c*u_b
        aux_v = v_a - c*v_b
        #Valor futuro d b
        aux_b = a - c*b
        #Hacemos los cambios
        u_a=u_b
        v_a=v_b
        u_b=aux_u
        v_b=aux_v
        a=b
        b=aux_b
        
    #a es el mcd, u el inverso si a es 1
    return [a,u_a, v_a]
#************************************************************
    
def inverso(a,n):
    """
    Algortimo para calcular el inverso modular de un número
    
    Entrada: 
        a y n enteros
    
    Salida: 
        El inverso modulo n de a: a^-1 mod n
    """
    d_u=mcd(a,n)
    if (d_u[0]!=1):
        print "No existe el inverso modulo "+str(n)+ " para "+str(a)
    else:
        #1=u*a+v*n -> u es el inverso
        return d_u[1]%n
        
#***********************************************************

La función de cifrado, que como se ha dicho es la sumatoria del producto de cada bit del mensaje por cada elemento de la llave pública:

In [2]:
def cifrado(m,kpub):
    """
    Cifrado de knapstack
    
    Entrada:
        m: mensaje a cifrar formado por lista binaria de 1 y 0
        
        kpub: secuencia obtenida a partir de una supercreciente[a1*...ak*] haciendo u*kpriv[i]modn
        
    Salida:
        sum: mensaje cifrado
    """
    sum = 0
    for i in range(len(m)):
        sum = sum + m[i]*kpub[i]
    
    return sum

Para poder descifrar, hay que implementar el algoritmo avaricioso. Este algoritmo, dado un número, busca los elementos de una lista supercreciente que al sumarlos dan dicho número.

In [3]:
def greedy(n,s):
    """
    Algoritmo avaricioso
    
    Entrada:
        n: número del cual quiere conocerse sus sumandos en la lista s
        
        s: secuencia supercreciente
    
    Salida:
        sum: lista que contiene los elementos de s que al sumar dan n
    """
    
    suma = []
    #encontramos el primer elemento de la mochila que sea anterior a n
    for i in range(len(s)):
        if (s[i] >= n):
            suma.append(s[i-1])
    if (len(suma)==0):
        suma.append(s[len(s)-1])
    
    j=0
    fin = False
    aux= n- suma[0]
    while not fin:
        for i in range(len(s)):
            #si s[i] es mayor que aux, nos quedamos con s[i-1]
            #siempre y cuando s[i-1] no esté ya en suma y no sea 1
            if (s[i] >= aux and s[i-1] not in suma and aux!=1 and s[i-1]<aux):
                suma.append(s[i-1])
                j = j+1
                aux = aux - suma[j]
                if (aux in s):
                    suma.append(aux)
                if (aux ==1): #si aux ya es 1, paro
                    fin = True
        
        #Comprobamos si la suma de los elementos de la lsita sum da n
        if (sum(suma)==n):
            fin = True
        
    return suma

Para descifrar, calculamos v = inverso de u mod n. Entonces calculamos b=vb* mod n y tenemos:

b = vb* mod n ; b asterisco es el mensaje cifrado

  = v · Σe<sub>i</sub>a<sub>i</sub>* ; a<sub>i</sub> asterisco es la lista supercreciente modificada y e<sub>i</sub> el mensaje a cifrar
  
  = v · Σe<sub>i</sub>(ua<sub>i</sub>) mod n ; a<sub>i</sub> lista supercreciente
  
  = (uv) · Σe<sub>i</sub>a<sub>i</sub>
  
  = Σ e<sub>i</sub>a<sub>i</sub> mod n

Al final el descifrado se convierte en buscar los sumandos de b en la lista supercreciente.

In [10]:
def descifrado(c, kpriv):
    """
    Descifrado de knapstack
    
    Entrada:
        c: mensaje cifrado
        
        kpriv: formada por una secuencia super creciente [a1...ak], n y u tal que gcd(n,u)=1
        
    Salida:
        d: lista binaria del mensaje descifrado
    """    
    a = kpriv[0] #la secuencia supercreciente
    n = kpriv[1]
    u = kpriv[2]
    
    v = inverso(u,n)
    
    b = (v*c) % n #b  = Sum m[i]*a[i]
   
    #descomponemos b como suma de elementos de la mochila
    lista =  greedy(b,a)
    
    #Expresamos los elementos de la mochila como binario
    d = []
    for i in a:
        if i in lista: #si el elemento de lista está en mochila, ponemos 1
            d.append(1)
        else:
            d.append(0)
    
    return d

Una prueba:

In [13]:
m = [0,1,1,0,0,1,0,1]
a = [1,3,7,15,31,63,127,255]
n = 557
u = 323
print mcd(n,u)[0]
kpriv = [a,n,u]
kpub=[] #kpub = a*u mod n
for i in a:
    kpub.append((i*u)%n)
    
c = cifrado(m, kpub)
d = descifrado(c, kpriv)
print "mensaje original", m
print "mensaje descifrado", d

1
mensaje original [0, 1, 1, 0, 0, 1, 0, 1]
mensaje descifrado [0, 1, 1, 0, 0, 1, 0, 1]


### Ejercicio 2: Logaritmo discreto

Hay que calcular el inverso de nuestra fecha de nacimiento dada la función Z<sub>p</sub> -> Z<sub>p</sub>, x -> alfa<sup>x</sup>, donde alfa es un elemento primitivo de Zp*.

Antes cargamos algunas funciones de la práctica 1 que nos hacen falta:

In [54]:
from random import randrange,randint

#********************************************************
def expmod(b,e,m):
    """
    Calcula la potencia modular de un número
    
    Entrada:
        b: base entera
        
        e: exponente entero
       
        m: modulo entero
    
    Salida:
        prod: resultado de (b^e) mod n
    """
    #b = base
    #e = exponente
    #m = modulo
    s=e
    prod=1 #En prod voy almacenando el resultado de las potencias
    a=b
    
    while(s>0):
        #Caso 1: primer bit es 1
        if(s%2==1):
            #Cambio prod
            prod = (prod * a) % m
            #Cambio s
            s = (s-1)/2
        #Caso 2: primer bit es 0
        elif (s%2==0):
            #No cambio prod
            #Cambio s
            s = s / 2
        #Cambio a
        a = (a*a) % m
        if(a==1):
            return prod
    
    #Devuelvo el resultado:
    return prod
#********************************************************
def millerrabin(p):
    """
    Implementación del algoritmo de mille rabin para comprobar si un número
    es primo
    
    Entrada:
        p: número candidato
        
    Salida:
        True: Si es probablemente primo
        
        False: Si no es primo
    """
    
    #Expresamos p-1 como (2**u)*s
    #Calculamos s y u
    s = p-1
    u=0
    while (s%2==0): #vemos si es primo
        s = s//2
        u=u+1
    
    
    #Escogemos un a aleatorio entre 2 y p-2 Unif[2,p-2] p-2>2
    #randrange(inicio, fin, salto)
    #inicio: se incluye en el rango
    #fin: fin del rango, no se incluye en el rango
    a = randrange(2,p-1) 
       
    #Si a**s=1 o a**s=-1
    if(expmod(a,s,p)==1 or expmod(a,s,p)==p-1): #p-1 es -1 en mod p
        return True
    else:
        #Desde i=1 hasta u-1
        for i in range(1,u, 1):
            #Si a^((2^k)*u) igual a -1 con k<u -> probablemente primo
            exp=(2**i)*s
            if (expmod(a,exp,p)==p-1): 
                return True
            #Si a^((2^k)*u) igual a 1 no precedido de -1 -> no primo
            elif (expmod(a,exp,p)==1):
                return False
        #Si no aparece el 1-> No primo
        return False
        
#*****************************************************************************
def esprimo(p,n):
    """
    Función que pasa el test de miller rabin varias veces, y dice si es primo
    o no con un porcentaje
    
    Entrada: 
        p: número que pasa test
        
        n: número de veces que se hace el test
    
    Salida: 
        True si primo, False si no primo
    """
    siprimo=0   
    for i in range (1,n,1):
        primo = millerrabin(p)
        if (primo==True):
            siprimo=siprimo+1
        else:
            #print "No es primo"
            return False
    #Calculamos porcentajes:
        
    porcentajeprimo = (float(siprimo)/n)*100
    #Además, calculamos la probabilidad de que sea primo
    prob = ((4**n-1)/float(4**n))*100
    
    
    return True
    
#*****************************************************************************
def jacobi(a,p):
    """
    Extensión del simbolo de legendre para p no primo

    Entrada:
        a: entero
        p: impar
        
    Salida:
        1 si a es residuo cuadrático mod p
        
        0 si p divide a a
        
        -1 en caso contrario
        
    """   
    if(p!=1):
       a = a%p
    else: #si (a/1)
        return 1
    
    #Comprobamos los casos base
    if (a==0):
        return 0
    if (a==1):
        return 1
    if (a==2):
        if ( p%8 == 1 or p%8 == 7):
            return 1
        else:
            return -1
    if (a == p-1):
        if (p%4 == 1):
            return 1
        if (p%4 == 3):
            return -1
    
    #saco u y s
    s = a
    u=0
    while (s%2==0): #vemos si es par
        s = s//2
        u=u+1
        
    #Reciprocidad cuadrático
    signo1=1
    signo2=1
    if (u%2==0):
        signo1 = 1
    else:
        if (p%8 == 1 or p%8 == 7):
            signo1 = 1
        else:
            signo1 = -1
    
    if (p%4==1 or s%4==1):
        signo2 = 1
    elif (p%4==3 and s%4==3):
        signo2 = -1
    #Damos la vuelta        
    a = p
    p = s
    
    return signo1*signo2*jacobi(a,p)
#*****************************************************************************
def petitpasgrandpas(a,b,p):
    """
    Implementación del algoritmo paso enano-paso gigante
    Calcula x tal que a^x = b mod p
    
    Entrada: 
        a, b: enteros
        
        p: primo
    
    Salida: 
        x tal que a^x= b mod p si hay solución 
        
        False si no hay solución
    
    """
    #Calculamos s: primer primo que elevado a 2 da > que p-1
    s=1
    while(s**2<p-1):
        s=s+1
    
    #Sacamos la lista grande y la pequeña
    L = [expmod(a,s*i,p) for i in range(1,s+1)]
    l = [(b*expmod(a,i,p))%p for i in range(s)]
    
    #Buscamos la insterseccion entre L y l
    n = list(set(L) & set(l))
    
    #Si no hay intersección, no hay solución
    if not n:
        print "No hay solución"
        return False
    
    #x = i*b - j
    #i indice de n en L +1
    #j indice de n en l
    i = L.index(n[0]) + 1
    j = l.index(n[0])
    
    x = i*s -j
    
    return x
#*****************************************************************************

Ya estamos en disposición de implementar lo que nos piden en el ejercicio. La función busca un elemento primitivo alfa de Zp, con p el primer primo mayor que mi DNI. Un elemento alfa será primitivo si el símbolo de jacobi de alfa sobre p es -1. Después, calcula la inversa de mi fecha de nacimiento usando el logaritmo discreto, que es la inversa de la función que proponen.

In [30]:
def primitivo(fecha):
    """
    Función que, dada la función Zp->Zp x->a^x,
    calcula el inverso de mi fecha de nacimiento
    
    Entrada:
        fecha: El valor de la fecha de nacimiento en formato AAAMMDD
    
    Salida: El inverso de la fecha de nacimiento
    """
    #Encuentra un elemento primitivo alfa de Zp
    #n es fecha de nacimiento
    
    #Buscamos un pseudoprimo mayor o igual que mi DNI
    p = 77378395
    
    primo = False
    while not primo:
        if (esprimo(p,10) and esprimo((p-1)/2,10) ):
            primo = True
        else:
            p = p+2
    
    #Búsqueda del elemento primitivo    
    primitivo = False
    alfa = 2
    while not primitivo:
        if (jacobi(alfa,p) == -1):
            primitivo = True
        else:
            alfa = alfa +1
    #La inversa de la función es el logaritmo discreto
    inv = petitpasgrandpas(alfa,fecha,p)
    return inv,alfa,p

Hagamos una prueba: 

In [33]:
fecha = 19930530
inv,alfa,p = primitivo(fecha)
print inv,alfa,p

23836488 2 77379227


El primer número es el inverso, el segundo el alfa primitivo y el tercero el p primo. Para comprobar que está bien, solo hay que aplicar la función al resultado, es decir, hacer alfa<sup>23836488</sup> mod p

In [34]:
print expmod(alfa,inv,p)

19930530


Como vemos se obtiene la fecha de nacimiento por lo que el inverso es correcto.

p y q son enteros primos y n = pq en lo que sigue.

### Ejercicio 3: Función de Rabin

In [40]:
def rabin():
    """
    Función que calcula los factores primos de un entero y los devuelve
    """
    
    #mcd(x-y,n) es un divisor no trivial de n

    p = mcd(48478872564493742276963,37659670402359614687722+12)[0]
    q = mcd(48478872564493742276963,37659670402359614687722-12)[0]
    
    return p,q   

In [41]:
p,q = rabin()
print p,q
print p*q #Comprobación de que son divisores de n 

303948303229 159497098847
48478872564493742276963


### Ejercicio 4: Función resumen

Usando la función h definica por Goldwasser, Micali y Rivest y la construcción de Merkle Damgard, implementar una función resumen. a0, a1 y n son públicos. Cogemos la misma n que en el ejercicio anterior.

In [42]:
def h(x,b):
    """
    Función de compresión 
    
    """
    n = 48478872564493742276963
    #a0 y a1 dos cuadrados arbitrario mod n
    a0 = (6347823468234**2) % n
    a1 = (5143151387122**2) % n
    
    return ((x**2)  * (a0**b) * (a1 **(1-b))) % n

In [43]:
def resumen(x,m):
    """
    Función resumen que usa una estructura de merkle damgard usando h como
    función de compresión
    
    Entrada:
        x: vector inicial en forma de entero
        m: mensaje al que aplicar el resumen en forma de lista de bits
    
    Salida:
        salida: el resumen aplicado a m en forma de entero
    """
    #x vector inicial, m mensaje al que aplicar el hash
    
    salida = 0
    for i in range(0,len(m)):
        #Construccion merkle damgard
        #La salida de la h se combina en una nueva h con
        #el nuevo bloque de mensaje que es m[i]
        salida = h(x,m[i]) #m[i] es b 
        x = salida #lo que sustituye al vector inicial en los sucesivos pasos es la salida de h
    
    return salida

In [44]:
print resumen(7687,[1,0,1,1])

7682878834527310956964


### Ejercicio 5: Inverso RSA

Dada la función RSA x<sup>e</sup> mod p, con p el primer primo mayor que mi DNI, nos piden calcular el inverso de la función para 1234567890.

In [49]:
def inversorsa(p,q,y):
    """
    Función que calcula el inverso de la función RSA f : Zn -> Zn, x -> x^e .
    
    Entrada:
        p: numero de identidad
        q: fecha de nacimiento
        y: el valor al que calcular el inverso
    Salida:
        x: el inverso de la función RSA para el valor y
    """
    #pprimo el menor primo entero mayor que mi numero de identidad
    pprimo = False
    #qprimo el primer primo mayor que mi fecha de nacimiento
    qprimo = False
    
    while not pprimo:
        if (esprimo(p,10)):
            pprimo = True
        else:
            p = p+1
    
    while not qprimo:
        if (esprimo(q,10)):
            qprimo = True
        else:
            q = q+1
    
    n = p*q
    
    e = 2
    phi = (p-1)*(q-1)
    while (mcd(e,phi)[0] !=1 ):
        e = e+1
    
    #inverso de x^e=y es sacar x dado e y y
    
    d = inverso(e, phi)
    x = expmod(y,d,n)
    return x,e,n
    

In [50]:
p=77378395
q=19930530
y = 1234567890 #elemento al que calcular el inverso
print inversorsa(p,q,y)

(1427435933906955L, 5, 1542194849755207)


Si hacemos la función RSA al valor inverso deberíamos obtener 1234567890.

In [51]:
print expmod(1427435933906955,5,1542194849755207)

1234567890


### Ejercicio 6: Descomposición en factores

Dado n = 50000000385000000551, encontrar sus factores primos p y q por un método distinto al del ejercicio 3, sabiendo que una inversa de x<sup>5</sup> es x<sup>10000000074000000101</sup>.

In [52]:
def ej6(n,d,e):
    """
    Algoritmo para encontrar los factores primos de n, dados d y e
    
    Entrada:
        n: entero a factorizar
        d: llave pública de la función Td
        e: llave privada de la función inversa de Td, Te
    
    Salida:
        False: si no es capaz de encontrar los factores
        p y q: factores primos de n
    """
    #una inversa de Zn -> Zn, x -> x^5 es x -> x^10000000074000000101
    #Te inversa de Td mod n, si lo conocemos, podemos factorizar n

    #de -1 = 2^a * b
    #de-1 = de1
    de1 = d*e - 1
    
    #Expresamos d*e-1 como (2**a)*b con b impar
    #Calculamos a y b
    b = de1
    a=0
    while (b%2==0):
        b = b//2
        a=a+1
    
    
    while True: #repetimos con distintos x hasta encontrar factores no triviales
        
        #x aleatorio entre 1 y n
        x = randint(1,n-1)
        gcd = mcd(x,n)[0]
        
        if (gcd!=1): #hemos encontrado un factor->stop
            return gcd
        
        if (gcd==1):
            #y = x^b mod n
            y = expmod(x,b,n)
    
        for i in range(a): #hay que repetir a veces
            z = y
            y = expmod(y,2,n) # y = y^2 mod n
            
            if (y == n-1):
                #Fail
                return False
            if (y==1):
                #exito
                p = mcd(n,z+1)[0]
                q = mcd(n,z-1)[0]
                if(p!=n and p!=1):
                    #Acabamos solo si no son factores triviales
                    return p,q
    
    return False

In [55]:
n = 50000000385000000551
d = 5
e = 10000000074000000101
print ej6(589,7,13) #ejemplo libro
print ej6 (n,d,e)

(19, 31)
(5000000029L, 10000000019L)


In [56]:
print 19*31
print 5000000029L*10000000019L

589
50000000385000000551


Comparándolo con el método del ejercicio 3, observamos que para obtener los factores en el ejercicio 3 hemos necesitado el resultado de dos valores para la función de Rabin. Sin embargo, para obtener los factores en este ejercicio solo hemos necesitado la inversa de la función, es decir, la llave pública y privada. Normalmente no se tiene acceso tanto a la clave pública como a la privada, sin embargo, sí que es posible encontrar el resultado de dos valores para la función de Rabin.

### Ejercicio 7: Firma RSA

En este ejercicio implementamos una firma RSA, con su generador de claves y su verificador de firma. Las claves pública y privada generada se guardan en los ficheros pub y priv, respectivamente. El mensaje está en un fichero llamado mensaje y la firma generada se guarda en un fichero llamado firma. La firma se realiza sobre un resumen del mensaje.

In [57]:
from hashlib import sha224

In [58]:
def clavesRSA(p,q):
    """
    Generación de claves RSA. Las guarda en su correspondiente fichero.
    
    Entrada: 
        p,q: primos grandes y distantes entres sí
    
    Salida:
        e,n: clave publica
        d: clave privada
    """
    
    n = p*q
    phi = (p-1)*(q-1)
    
    e = 2
    while(mcd(phi,e)[0]!=1):
        e = e +1
    
    d = inverso(e,phi)
    
    #Abrimos con permisos de escritura los ficheros donde se guardarán las claves
    fpub = open("pub","w")
    fpriv = open("priv","w")
    
    #Escribimos en un fichero la clave publica y privada, separando e y d de n con un espacio
    fpub.write(str(e)+" ")
    fpub.write(str(n))
    
    fpriv.write(str(d)+" ")
    fpriv.write(str(n))
    
    return e,n,d

El algoritmo para firmar:

In [60]:
def firmaRSA(priv,m):
    """
    Firma RSA -> r^d mod n=f . Guarda la firma en un fichero
    
    Entrada:
        priv: nombre del fichero de la clave privada en string (d y n)
        m: nombre del fichero del mensaje a firmar en string
    Salida:
        f: firma del resumen del mensaje
    """
    #Abrimos el fichero de mensaje
    myfile = open(m)
    #Lo guardamos en una variable como string
    m = myfile.read()    
    #Calculamos un resumen del mensaje
    r = sha224(m).hexdigest()
    #Convertimos de hexadecimal a entero base 10 para poder operar con el resumen
    r = int("0x"+r,0)
    
    #Leemos la clave del fichero
    fpriv = open(priv)
    clave = fpriv.read().split(" ")
    d = int(clave[0])
    n = int(clave[1])
    
    
    #firma del resumen:
    f = expmod(r,d,n)
    
    ffirma = open("firma","w")
    ffirma.write(str(f))
    
    return f

Y para verificar la firma:

In [61]:
def verificacionRSA(pub,m,f):
    """
    Verificación de firma RSA-> f^e mod n=resumen mod n
    
    Entrada:
        pub: nombre del fichero de la clave publica en string (e y n)
        m: nombre del fichero del mensaje a verificar en string
        f: nombre del fichero de firma en string
    
    Salida:
        True: si la firma es correcta
        False si no
    """
    #Abrimos el fichero de mensaje
    myfile = open(m)
    #Lo guardamos en una variable como string
    m = myfile.read()
    #Calculamos un resumen del mensaje
    r=sha224(m).hexdigest()
    #Convertimos de hexadecimal a base 10 para poder comparar con v
    r = int("0x"+r,0)
    
    
    #Leemos la clave del fichero
    fpub = open(pub)
    clave = fpub.read().split(" ")
    e = int(clave[0])
    n = int(clave[1])
    
    #Leemos el fichero con la firma
    ffirma = open("firma")
    f = int(ffirma.read())
    
    #verificacion
    v = expmod(f,e,n)

    
    if(v==r%n): #si la verificacion coincide con el resumen mod n->True
        return True
    else:
        return False

Para generar las claves RSA usamos dos primos grandes y distantes entre sí. Lo siguiente genera las claves, las guarda en sus ficherso y firma el mensaje. Luego, verifica la firma.

In [64]:
clavesRSA(1081298104698286063813737967304568031406522676857739555339880517562925221530558524296599584286163751908713364829390795648074146197550782524900963175263757603219
,204616454475328391399619135615615385636808455963116802820729927402260635621645177248364272093977747839601125961863785073671961509749189348777945177811)

m = 'mensaje' #el nombre del fichero mensaje, para abrirlo después
priv = "priv" #el nombre del fichero de clave privada
pub = "pub" #el nombre del fichero de clave publica
f = "firma" #el nombre del fichero de la firma
firmaRSA(priv,m)
print verificacionRSA(pub,m,f)

True


Para comprobar que la verificación funciona, si editamos el fichero de firma y realizamos de nuevo la verificación, obtenemos un valor False.

In [63]:
print verificacionRSA(pub,m,f)

False
